In [ ]:
import random

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import shapely.geometry as geo
import shapely.affinity as aff
from descartes.patch import PolygonPatch

In [ ]:
random.seed(1618)

In [ ]:
colours = plt.cm.viridis(np.linspace(0,1,10))
golden_ratio = (1 + np.sqrt(5)) / 2

In [ ]:
bound = 4
body_buffer = 0.14
head_buffer = body_buffer * golden_ratio
head_stretch = 1.3

expected_x_bounds = np.array([-bound - body_buffer, bound + head_buffer])
length = np.diff(expected_x_bounds)
height = length / golden_ratio
desired_amplitude = height / 2 - body_buffer

def snake_wave_packet(x):
    x = np.array(x)
    return desired_amplitude * np.exp(-x**2/2) * np.sin(2 * np.pi * x) 

expected_y_bounds = np.ravel([-height/2, height/2])

assert np.diff(expected_x_bounds) / np.diff(expected_y_bounds) == golden_ratio

In [ ]:
epsilon = 0.1
x_init = bound - head_buffer * head_stretch

y_vals = snake_wave_packet([x_init, x_init + epsilon])
ydiff = np.diff(y_vals)
y_init = y_vals[0]

head_shift = head_buffer * head_stretch
hypot = np.sqrt(epsilon**2 + ydiff**2)
num_shifts = head_shift / hypot
neck_shring = 1

head_angle = np.arctan(ydiff / epsilon) / np.pi * 180
head_middle_x = x_init + epsilon * num_shifts * neck_shring
head_middle_y = y_init + ydiff * num_shifts * neck_shring

In [ ]:
tongue_length = 0.1
tongue_buffer = 0.02
fork_angles = 35

tongue_init_x = head_middle_x + epsilon * num_shifts
tongue_init_y = head_middle_y + ydiff * num_shifts

tongue_shift_num = tongue_length / hypot

tongue_shift_x = epsilon * tongue_shift_num
tongue_shift_y = ydiff * tongue_shift_num

tongue_fork_x = tongue_init_x + tongue_shift_x
tongue_fork_y = tongue_init_y + tongue_shift_y

tongue_base = geo.LineString(
    [(tongue_init_x, tongue_init_y), (tongue_fork_x, tongue_fork_y)]
).buffer(tongue_buffer)

tongue_base

tongue_top_fork = aff.rotate(tongue_base, fork_angles, origin=(tongue_init_x, tongue_init_y))
tongue_top_fork = aff.translate(tongue_top_fork, tongue_shift_x, tongue_shift_y)

tongue_bot_fork = aff.rotate(tongue_base, -fork_angles, origin=(tongue_init_x, tongue_init_y))
tongue_bot_fork = aff.translate(tongue_bot_fork, tongue_shift_x, tongue_shift_y)


tongue = tongue_base.union(tongue_top_fork).union(tongue_bot_fork)
tongue

In [ ]:
# head_angle

In [ ]:
# ydiff

In [ ]:
# np.arctan(ydiff / epsilon) / np.pi * 180

In [ ]:
point = geo.Point(head_middle_x, head_middle_y)
head = point.buffer(head_buffer)
head = aff.scale(head, head_stretch, 1, origin=point)
head = aff.rotate(head, head_angle, origin=point)
head = head.union(tongue)

head

In [ ]:
x = np.linspace(-bound, x_init, 1000)
y = snake_wave_packet(x)
centre_line = geo.LineString(zip(x, y))
centre_line

In [ ]:
body = centre_line.buffer(body_buffer)
body

In [ ]:
snake = body.union(head)
snake

In [ ]:
# def plot_bounds():
#     plt.plot([expected_x_bounds[0], expected_x_bounds[1]], [expected_y_bounds[0], expected_y_bounds[0]], 'k', linewidth=0.5)
#     plt.plot([expected_x_bounds[0], expected_x_bounds[1]], [expected_y_bounds[1], expected_y_bounds[1]], 'k', linewidth=0.5)
    
#     plt.plot([expected_x_bounds[0], expected_x_bounds[0]], [expected_y_bounds[0], expected_y_bounds[1]], 'k', linewidth=0.5)
#     plt.plot([expected_x_bounds[1], expected_x_bounds[1]], [expected_y_bounds[0], expected_y_bounds[1]], 'k', linewidth=0.5)

In [ ]:
xmin, ymin, xmax, ymax = snake.bounds

In [ ]:
number_of_stripes = 100

stripe_x = np.random.uniform(xmin, xmax, size=number_of_stripes)
stripe_thickness = np.random.uniform(body_buffer/2, body_buffer, size=number_of_stripes)
stripe_x_next = stipe_x + epsilon

stripe_y = snake_wave_packet(stipe_x)
stripe_y_next = snake_wave_packet(stripe_x_next)

gradient = (stripe_y_next - stripe_y) / (stripe_x_next - stipe_x)
perp = -1/gradient
# norm = np.sqrt(np.sum(perp**2, axis=1))
# perp = perp / norm[:, None]

norm = np.sqrt((epsilon * perp)**2 + epsilon**2)

x_shifts = epsilon / norm
y_shifts = epsilon * perp / norm

coords_stripe_a_x = stripe_x + x_shifts
coords_stripe_a_y = stripe_y + y_shifts
coords_stripe_a = zip(coords_stripe_a_x, coords_stripe_a_y)

coords_stripe_b_x = stripe_x - x_shifts
coords_stripe_b_y = stripe_y - y_shifts
coords_stripe_b = zip(coords_stripe_b_x, coords_stripe_b_y)

stripes = [
    geo.LineString([a, b]).buffer(thickness).intersection(snake)
    for a, b, thickness in zip(coords_stripe_a, coords_stripe_b, stripe_thickness)
]


stripes[0]

In [ ]:
number_of_stripes = 400

resampled = geo.LineString([
    centre_line.interpolate(i, normalized=True) 
    for i in np.linspace(0,1, number_of_stripes*10)])

coords = resampled.coords

x_coords = np.array(coords)[:,0]

In [ ]:



def create_stripe(snake, body_buffer):

    
    
    
    stripe_length = body_buffer * 0.5
    epsilon = 0.001
    xmin, ymin, xmax, ymax = snake.bounds
    
#     stripe_x = np.random.uniform(xmin, xmax)
    stripe_x = np.random.choice(x_coords)
    
    
    stripe_thickness = np.random.uniform(body_buffer/3, body_buffer*0.8)
    stripe_x_next = stripe_x + epsilon

    stripe_y = snake_wave_packet(stripe_x)
    stripe_y_next = snake_wave_packet(stripe_x_next)

    gradient = (stripe_y_next - stripe_y) / (stripe_x_next - stripe_x)
    perp = -1/gradient

    norm = np.sqrt((epsilon * perp)**2 + epsilon**2)

    x_shift = epsilon / norm * stripe_length
    y_shift = epsilon * perp / norm * stripe_length

    coords_stripe_a_x = stripe_x + x_shift
    coords_stripe_a_y = stripe_y + y_shift
    coords_stripe_a = (coords_stripe_a_x, coords_stripe_a_y)
#     print(coords_stripe_a)

    coords_stripe_b_x = stripe_x - x_shift
    coords_stripe_b_y = stripe_y - y_shift
    coords_stripe_b = (coords_stripe_b_x, coords_stripe_b_y)

    stripe = geo.LineString(
        [coords_stripe_a, coords_stripe_b]).buffer(stripe_thickness, cap_style=2)
    
    if stripe.intersects(snake):
        return stripe.intersection(snake)
    else:
        return create_stripe(snake, body_buffer)


stripes = [
    create_stripe(snake, body_buffer)
    for i in range(number_of_stripes)
]

stripe_colours = colours[np.random.choice(np.arange(len(colours)), size=len(stripes))]

In [ ]:


# stripe_thickness = np.random.uniform(body_buffer/2, body_buffer*5, size=number_of_stripes)
# stripe_length = body_buffer * 5

# resampled = geo.LineString([
#     centre_line.interpolate(i, normalized=True) 
#     for i in np.linspace(0,1, number_of_stripes)])

# gradient = np.diff(resampled.coords, axis=0)
# perp = -1/gradient

# norm = np.sqrt(np.sum(perp**2, axis=1))

# perp = perp / norm[:, None]

# coords_stripe_mid = resampled.coords[0:-1]
# coords_stripe_a = coords_stripe_mid - perp * stripe_length
# coords_stripe_b = coords_stripe_mid + perp * stripe_length

# stripes = [
#     geo.LineString([a, b]).buffer(thickness).intersection(snake)
#     for a, b, thickness in zip(coords_stripe_a, coords_stripe_b, stripe_thickness)
# ]

# stripe_colours = colours[np.random.choice(np.arange(len(colours)), size=len(stripes))]

# stripes[0]

In [ ]:
fig = plt.figure(1, figsize=(10,10), dpi=90)
fig.set_frameon(True)
ax = fig.add_subplot(111)

patch = PolygonPatch(snake, facecolor=colours[4])
ax.add_patch(patch)

plt.xlim(-bound-1, bound+1)
plt.ylim(-bound-1, bound+1)

to_be_shuffled = np.array(list(zip(stripes, stripe_colours)))
np.random.shuffle(to_be_shuffled)

for stripe, stripe_colour in to_be_shuffled:
    patch = PolygonPatch(stripe, facecolor=stripe_colour, edgecolor=stripe_colour)
    ax.add_patch(patch)

#     plot_bounds()

#     plt.plot(x,y, color='white')

plt.show()